In [12]:
from transformers import XLMRobertaTokenizer

# XLM-Roberta tokenizer yükleniyor
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Test cümlesi
text = "Merhaba dünya"

# Tokenize işlemi (tensor olarak, parçaları göstermeden)
encoded = tokenizer(text, return_tensors="pt")

# Token ID’leri
print("Input IDs:", encoded["input_ids"])

# İsteğe bağlı: Token sayısını da görmek istersen

print("Sequence Length:", encoded["input_ids"].shape[1])
print(tokenizer.tokenize("Selam dünya! # £& + -* / ., 🌍🔥 我 𓂀"))

Input IDs: tensor([[     0, 120906,  15027,      2]])
Sequence Length: 4
['▁Se', 'lam', '▁dünya', '!', '▁#', '▁£', '&', '▁+', '▁-', '*', '▁/', '▁', '.', ',', '▁', '🌍', '🔥', '▁我', '▁', '𓂀']


In [11]:
tokenizer.encode("Selam dünya! # £& + -* / ., 🌍🔥 我 𓂀")

[0,
 503,
 3719,
 15027,
 38,
 468,
 11762,
 1230,
 997,
 20,
 1639,
 248,
 6,
 5,
 4,
 6,
 247290,
 222326,
 13129,
 6,
 3,
 2]

In [1]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

# CSV'den veri yükle
dataset = Dataset.from_csv("../../../BKÜ Sınav Analizi - BKÜ Sınav Analizi.csv")
dataset_test = Dataset.from_csv("../../../tarim_veriseti_test.csv")

# Hugging Face Dataset'leri
dataset2 = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"Input": "Soru", "Output": "GPT"})
dataset3 = load_dataset("merve/turkish_instructions", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"talimat": "Soru", " giriş": "inputs"," çıktı":"GPT"})
dataset4 = load_dataset("TFLai/Turkish-Alpaca", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"instruction": "Soru", "input": "inputs","output":"GPT"})
dataset5 = load_dataset("umarigan/openhermes_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"instruction": "Soru", "input": "inputs","output":"GPT"})
datasetWiki = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Soru", "answer": "GPT"})


def replace_empty_with_none(example):
    # 'inputs' sütunundaki boş karakteri None ile değiştirelim
    if example['inputs'] == "":
        example['inputs'] = None
    return example


# dataset4'teki 'inputs' sütunundaki boş karakterleri None ile değiştir


dataset4 = dataset4.map(replace_empty_with_none)

# dataset5'teki 'inputs' sütunundaki boş karakterleri None ile değiştir
dataset5 = dataset5.map(replace_empty_with_none)

# Tüm veri setlerini birleştir
all_datasets = concatenate_datasets([dataset, dataset2, dataset3, dataset4, dataset5, datasetWiki, dataset_test])

all_datasets = all_datasets.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)
all_datasets = all_datasets.rename_columns({"Soru": "instruction", "GPT": "output"})
all_datasets = all_datasets.remove_columns(['id', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx'])




# Yazılacak dosya
with open("metin.txt", "w", encoding="utf-8") as f:
   
        for example in all_datasets:
            for key in example:
                val = example[key]
                if isinstance(val, str):
                    val = val.strip()
                    if len(val) > 0:
                        f.write(val + "\n")


Filter:   0%|          | 0/2952594 [00:00<?, ? examples/s]

In [2]:
special_tokens = [
    "<s>", "</s>", "<unk>", "<pad>",                 # Temel tokenler
    "<|system|>", "<|user|>", "<|assistant|>", "<|end|>",  # Chat rolleri
    "<function>", "</function>",                    # Function calling
    "<function_name>", "</function_name>",
    "<arguments>", "</arguments>",
    "<result>", "</result>"
]


In [3]:
tokenizer.train(files=["metin.txt"], vocab_size=30_000, min_frequency=2, special_tokens=special_tokens)

In [4]:
tokenizer.decode(tokenizer.encode("Selam dünya! 🌍🔥1500.,").ids)

'Selam dünya! 🌍🔥1500.,'

In [5]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parameters',
 '_tokenizer',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'from_file',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalize',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'save_model',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [6]:
tokenizer.token_to_id("<unk>")

2

In [23]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parameters',
 '_tokenizer',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'from_file',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalize',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'save_model',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [7]:
import emoji

# Unicode'daki tüm emojileri içeren liste
new_tokens = list(emoji.EMOJI_DATA.keys())

# Tokenizer'a yeni tokenları ekle
tokenizer.add_tokens(new_tokens)

# Güncellenmiş tokenizer'ı kaydet
#tokenizer.save_pretrained("./updated_tokenizer")

5042

In [8]:
tokenizer.decode(tokenizer.encode("Selam dünya! 🌍🔥").ids)

'Selam dünya! 🌍🔥'

In [9]:
print(tokenizer.encode("Selam dünya! # £& + -* / ., 🌍🔥 我").tokens)

['Selam', 'ĠdÃ¼nya', '!', 'Ġ#', 'ĠÂ£', '&', 'Ġ+', 'Ġ-', '*', 'Ġ/', 'Ġ.', ',', 'Ġ', '🌍', '🔥', 'Ġ', 'æ', 'Ī', 'ĳ']


In [10]:
tokenizer.get_vocab_size()

35040

In [11]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        unk_token="<unk>",
        pad_token="<pad>",
        bos_token="<s>",
        eos_token="</s>",
    )

# Padding token ID’sini sıfıra ayarlıyoruz

tokenizer.pad_token = "<pad>"
tokenizer.add_special_tokens({'pad_token': '<pad>'})

tokenizer.save_pretrained("../CrispyTokenizer")

('../CrispyTokenizer/tokenizer_config.json',
 '../CrispyTokenizer/special_tokens_map.json',
 '../CrispyTokenizer/tokenizer.json')

In [12]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_call_one',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_pad',
 '_pad_token_type_id',
 '_processor_class',
 '_save_pretrained',
 '_set_model_sp

In [13]:
from transformers import PreTrainedTokenizerFast


# ✅ Test cümlesi: özel tokenlar + bilinmeyen sembol + normal cümle
text = "Merhaba 𓂀 dünya"

# 🔄 Tokenizasyon
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
decoded = tokenizer.decode(ids, skip_special_tokens=False)

# 📊 Çıktılar
print("🔹 Tokenler:")
print(tokens)
print("\n🔹 Token ID'leri:")
print(ids)
print(f"\n🔹 Decode sonucu:\n{decoded}")


🔹 Tokenler:
['Merhaba', 'Ġ', 'ð', 'ĵ', 'Ĥ', 'Ģ', 'ĠdÃ¼nya']

🔹 Token ID'leri:
[6835, 236, 188, 257, 240, 238, 2414]

🔹 Decode sonucu:
Merhaba 𓂀 dünya


In [14]:
print(tokenizer.decode(tokenizer.encode("Selam dünya! # £& + -* /  🌍🔥 我"), skip_special=False ) )

Selam dünya! # £& + -* /  🌍🔥 我


In [33]:
text = "Merhaba 𓂀 dünya"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("Token IDs:", ids)
print("UNK token ID:", tokenizer.unk_token_id)


Tokens: ['M', 'er', 'h', 'aba', 'Ġ', 'ð', 'ĵ', 'Ĥ', 'Ģ', 'ĠdÃ¼nya']
Token IDs: [60, 276, 87, 1365, 236, 188, 257, 240, 238, 982]
UNK token ID: 2


POSİTİON İDS

In [34]:
import torch


def get_position_ids(input_ids):
    # input_ids: (batch_size, seq_len)
    seq_len = input_ids.size(1)
    return torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand_as(input_ids)

get_position_ids(torch.tensor(([tokenizer.encode("Selam dünya! 🌍🔥").ids])))

AttributeError: 'list' object has no attribute 'ids'